In [1]:
from os import listdir
import pandas as pd

for filename in listdir('output'):
    if '_grid_search' in filename:
        print(filename)

xgb_ordinal2_grid_search.csv
lgb_ordinal2_grid_search.csv
catb_ordinal2_grid_search.csv
nn_ordinal2_grid_search.csv


In [2]:
gs_xgb_res = pd.read_csv('output/xgb_ordinal2_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
#0.829998
gs_xgb_res.head()

,alpha,best_round,booster,colsample_bylevel,colsample_bytree,cv,data_name,early_stopping_rounds,eta,eval_metric,...,objective,pred_timespent,scale_pos_weight,seed,subsample,timestamp,train_all_auc,train_auc,tree_method,val_auc
fpLT,0,7,gbtree,0.675,0.7,False,ordinal2,1,0.1,auc,...,binary:logistic,0:00:00,1,40344667,0.7,2018-07-25 16:07:39,0.721899,-1,hist,0.711196
8erP,0,4,gbtree,0.675,0.7,False,ordinal2,1,0.1,auc,...,binary:logistic,0:00:00,1,40853299,0.7,2018-07-25 16:07:39,0.713335,-1,hist,0.705814


In [3]:
gs_catb_res = pd.read_csv('output/catb_ordinal2_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_AUC'], ascending=False)
gs_catb_res.head()

,best_round,cv,data_name,depth,eval_metric,gs_timespent,iterations,l2_leaf_reg,logging_level,pred_timespent,random_seed,thread_count,timestamp,train_AUC,train_all_AUC,val_AUC
Val5,30,False,ordinal2,8,AUC,0:00:03,30,0.7,Verbose,0:00:03,33892013,15,2018-07-25 16:07:32,0.707702,0.722036,0.699959
yp1r,30,False,ordinal2,9,AUC,0:00:02,30,2.2,Verbose,0:00:04,26076149,15,2018-07-25 16:07:25,0.704676,0.727607,0.698954


In [4]:
gs_lgb_res = pd.read_csv('output/lgb_ordinal2_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
#0.829998
gs_lgb_res.head()

,bagging_fraction,bagging_freq,bagging_seed,best_round,boosting,cv,data_name,early_stopping_round,feature_fraction,feature_fraction_seed,...,num_boost_round,num_leaves,num_threads,objective,pred_timespent,scale_pos_weight,timestamp,train_all_auc,train_auc,val_auc
DQ7E,0.9,1,24200148,30,gbdt,False,ordinal2,10,1.0,24200148,...,30,31,8,binary,0:00:00,1,2018-07-25 16:07:23,0.798216,0.803446,0.730565
wTAx,0.9,1,25274724,30,gbdt,False,ordinal2,10,1.0,25274724,...,30,15,8,binary,0:00:00,1,2018-07-25 16:07:24,0.765827,0.767328,0.727868


In [5]:
gs_nn_res = pd.read_csv('output/nn_ordinal2_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
gs_nn_res.head()

,batch_size,best_epoch,cat_emb_drop_rate,cat_emb_outdim,combined_layers_dense_units,combined_layers_drop_rate,data_name,ep_for_lr,gs_timespent,int_list,...,patience,pred_batch_size,pred_timespent,timestamp,train_all_auc,trn_acc,trn_loss,val_acc,val_auc,val_loss
RStN,256,2,0.2,30,"""[100, 50]""",0.1,ordinal2,2,0:00:11,"""['num_layers_dense_units', 'combined_layers_d...",...,1,5000,0:00:00,2018-07-25 16:07:41,-1,0.813647,0.438728,0.806933,0.732794,0.442593
TN9s,512,4,0.1,30,"""[100, 50]""",0.1,ordinal2,2,0:00:10,"""['num_layers_dense_units', 'combined_layers_d...",...,1,5000,0:00:00,2018-07-25 16:07:54,-1,0.816494,0.431491,0.807733,0.732308,0.442544


In [6]:
stacknet_res = pd.read_csv('output/oof/stacknet_report.csv').sort_values(by=['oof_cv_score'])
stacknet_res['num_model_data'] = stacknet_res['chosen model_data'].apply(lambda x: len(x.split('|')) if x is not np.nan else 0)
stacknet_res

,model_data,gs_val_auc,test_score,oof_cv_score,chosen model_data,num_model_data
3,Val5__CATB_ordinal2_layer1,0.699959,0.707164,0.700463,NaN,0
4,fpLT__XGB_ordinal2_layer1,0.711196,0.700778,0.705077,NaN,0
2,RStN__NN_ordinal2_layer1,0.732794,0.724357,0.721945,NaN,0
1,DQ7E__LGB_ordinal2_layer1,0.730565,0.719489,0.724002,NaN,0
0,4TYq__NN_layer2,NaN,0.727489,0.726530,fpLT__XGB | DQ7E__LGB | Val5__CATB | RStN__NN,4
5,tTgj__LOGREG_layer2,NaN,0.725496,0.727630,fpLT__XGB | DQ7E__LGB | Val5__CATB | RStN__NN,4


In [ ]:
# 1. oof_cv_score vs. gs_val_auc
# for NN, oof_cv_score != gs_val_auc is normal, because gs_val_auc is get by train+val in grid search, while 
# oof_cv_score is got by nfolds cv
# for LGB, if gs_cv is true in config file, then oof_cv_score == gs_val_auc (almost). otherwise, based on the same
# reason as NN, oof_cv_score != gs_val_auc is normal 

# 2. test_score
# for NN: the one(not showing in log) is got by train+val and load_weights when doing prediction after grid search
# (when do_preds is true in config file), and test_score is got during oof where all trainig data are used. It's 
# normal that they are not equal
# for LGB, since model is retrain using all training data when doing prediction after grid search, the one is not in 
# log == test_score

In [ ]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload
%autoreload 2

In [ ]:
from automl_libs import BaseLayerResultsRepo

In [ ]:
repo = BaseLayerResultsRepo(['label'], 'output/oof/', load_from_file=True)

In [ ]:
for m,s in repo.show_scores():
    print(s,m)

In [ ]:
gs_nn_res.head(0).T.to_dict()

In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
lsvc = CalibratedClassifierCV(LinearSVC())

In [ ]:
lsvc.fit(np.array([0,1,2,3,4,5]).reshape(-1,1), [1,0,0,1,1,0])

In [ ]:
lsvc.predict_proba(2.22)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
svc = SVC(kernel='linear', probability=True)#, class_weight='auto')


In [ ]:
svc.fit(np.array([1,2,3,4,5]).reshape(-1,1), [0,0,1,1,0])

In [ ]:
svc.predict_proba(2.22)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
clf = OneVsRestClassifier(BaggingClassifier(svc, max_samples=1.0 / 5, n_estimators=5))
clf.fit(np.array([1,2,3,4,5]).reshape(-1,1), [0,0,1,1,0])

In [ ]:
clf.predict_proba(2.22)